In [19]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from datasets import load_dataset

# load model and processor
processor = WhisperProcessor.from_pretrained("openai/whisper-large")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-large")
model.config.forced_decoder_ids = None

"""
# load dummy dataset and read audio files
ds = load_dataset("hf-internal-testing/librispeech_asr_dummy", "clean", split="validation")
sample = ds[0]["audio"]
input_features = processor(sample["array"], sampling_rate=sample["sampling_rate"], return_tensors="pt").input_features 

# generate token ids
predicted_ids = model.generate(input_features)
# decode token ids to text
transcription = processor.batch_decode(predicted_ids, skip_special_tokens=False)

transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)
"""

'\n# load dummy dataset and read audio files\nds = load_dataset("hf-internal-testing/librispeech_asr_dummy", "clean", split="validation")\nsample = ds[0]["audio"]\ninput_features = processor(sample["array"], sampling_rate=sample["sampling_rate"], return_tensors="pt").input_features \n\n# generate token ids\npredicted_ids = model.generate(input_features)\n# decode token ids to text\ntranscription = processor.batch_decode(predicted_ids, skip_special_tokens=False)\n\ntranscription = processor.batch_decode(predicted_ids, skip_special_tokens=True)\n'

In [4]:
import torch
import torchaudio
from transformers import WhisperProcessor, pipeline
from datasets import load_dataset
import evaluate
import tempfile
import os

# Step 1: Load the common_voice dataset for Tamil
dataset = load_dataset("mozilla-foundation/common_voice_11_0", "ta", split="test")


The repository for mozilla-foundation/common_voice_11_0 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mozilla-foundation/common_voice_11_0.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


Generating train split: 0 examples [00:00, ? examples/s]


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 10023it [00:00, 100223.40it/s]
Reading metadata...: 20137it [00:00, 100748.63it/s]
Reading metadata...: 30401it [00:00, 101605.16it/s]
Reading metadata...: 41710it [00:00, 100095.10it/s]


Generating validation split: 0 examples [00:00, ? examples/s]


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 11758it [00:00, 92884.10it/s][A


...


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 11815it [00:00, 101623.33it/s]


Generating other split: 0 examples [00:00, ? examples/s]


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 9065it [00:00, 90643.39it/s]
Reading metadata...: 19079it [00:00, 96226.68it/s]
Reading metadata...: 30105it [00:00, 102629.96it/s]
Reading metadata...: 41222it [00:00, 105997.07it/s]
Reading metadata...: 51822it [00:00, 105363.91it/s]
Reading metadata...: 62360it [00:00, 103454.27it/s]
Reading metadata...: 72712it [00:00, 102631.54it/s]
Reading metadata...: 87993it [00:00, 100637.63it/s]


Generating invalidated split: 0 examples [00:00, ? examples/s]


Reading metadata...: 5575it [00:00, 91799.80it/s]


In [5]:
dataset[0]["audio"]

{'path': '/root/.cache/huggingface/datasets/downloads/extracted/256dec25b50e8072243364b2a76958f3978dd0a3272161faa2272dcf50c8fa20/ta_test_0/common_voice_ta_26622621.mp3',
 'array': array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        3.34414472e-05, 2.36578981e-05, 1.05088511e-05]),
 'sampling_rate': 48000}

In [6]:
import torchaudio

def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = torchaudio.load(batch["path"])
    batch["speech"] = speech_array[0].numpy()
    batch["sampling_rate"] = sampling_rate
    batch["target_text"] = batch["sentence"]
    return batch


In [7]:
dataset = dataset.map(speech_file_to_array_fn, remove_columns=dataset.column_names)

Map:   0%|          | 0/11815 [00:00<?, ? examples/s]

In [10]:
import librosa
import numpy as np

def resample(batch):
    speech_array = np.asarray(batch["speech"])
    batch["speech"] = librosa.resample(speech_array, orig_sr=48_000, target_sr=16_000)
    batch["sampling_rate"] = 16_000
    return batch

In [9]:
dataset = dataset.map(resample, num_proc=4)

/opt/conda/lib/python3.10/site-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=4):   0%|          | 0/11815 [00:00<?, ? examples/s]

In [13]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [16]:
processor = WhisperProcessor.from_pretrained("openai/whisper-large")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-large")
model.config.forced_decoder_ids = processor.get_decoder_prompt_ids(language="ta", task="transcribe")

model.to(device)


WhisperForConditionalGeneration(
  (model): WhisperModel(
    (encoder): WhisperEncoder(
      (conv1): Conv1d(80, 1280, kernel_size=(3,), stride=(1,), padding=(1,))
      (conv2): Conv1d(1280, 1280, kernel_size=(3,), stride=(2,), padding=(1,))
      (embed_positions): Embedding(1500, 1280)
      (layers): ModuleList(
        (0-31): 32 x WhisperEncoderLayer(
          (self_attn): WhisperSdpaAttention(
            (k_proj): Linear(in_features=1280, out_features=1280, bias=False)
            (v_proj): Linear(in_features=1280, out_features=1280, bias=True)
            (q_proj): Linear(in_features=1280, out_features=1280, bias=True)
            (out_proj): Linear(in_features=1280, out_features=1280, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1280, out_features=5120, bias=True)
          (fc2): Linear(in_features=5120, out_features=1280, bias

In [20]:
def transcribe(batch):
    input_features = processor(batch["speech"], sampling_rate=batch["sampling_rate"], return_tensors="pt").input_features 

    # generate token ids
    with torch.no_grad():
        predicted_ids = model.generate(input_features)
    # decode token ids to text
    #transcription = processor.batch_decode(predicted_ids, skip_special_tokens=False)
    #print(transcription)
    batch["transcription"] = processor.batch_decode(predicted_ids, skip_special_tokens=True)
    #print(transcription)
    return batch

In [ ]:
first_10_records = dataset[:10]

In [ ]:
dataset.map(transcribe)